In [2]:
import numpy as np
import itertools
from random import randint
from IPython.display import clear_output
from scipy.stats import entropy
from scipy.special import comb
from math import log, factorial, floor, ceil, isinf, isnan

from LPFullBib import CenteredBinomialFullBib as binomialbib #syntax binomialbib[eta, level]
from LPHalfBib import CenteredBinomialHalfBib as binomialbib_rep2
#centered binomial distribution is called with etaSearch(eta)

from May_Results import Maydist, Maybib
from May_Results import ternarybib #syntax ternarybib[omega, level]
#Ternary distribution with May's omega values is called with Maydist[i], 0<=i<=5

from Uniformbibs import uniformoptibib as uniformbib #syntax uniformbib[eta, level]
#Uniform distribution is called with uniSearch(eta)

q=3329

# distributions are denoted with arrays of the form [w_0,w_1,...,w_eta].
# distributions need to be of length at least 5, filled up w/ zeroes if necessary.

# parameter sets are stored as dictionaries (referred to as bibs). For example, To call upon
# level 3 parameter epsilon_20^{(3)}, you call Bib(20, 3).
# If Bib is a Rep-2 parameter set, Bib(30, j), Bib(31, j), Bib(32, j), Bib(33, j)
# exist but are constantly set to 0.0 .

# Each set of parameters can be called upon by utilizing their respective syntax.
# The respective syntaxes differ, depending on the distribution.

# To evaluate a certain distribution "dist" with parameter set "bib" for level "j", call
# for runtimes: Evaluate_T(dist, j, bib)
# for memory size: Evaluate_L(dist, j, bib)
# the returned array gives you the calculated values PER LEVEL, i.e. 
# Evaluate_T(dist, j, bib)[0] returns the calculated runtime on level 0

# For example, to verify our results for B(3), call
# Evaluate_T(etaSearch(3),7,binomialbib[3, 7])

In [3]:
def peta(eta,Permaway=[]):
    # Probability for an randomly drawn entry to be in the permaway-set
    sum=0
    for i in Permaway:
        if i==0:
            sum=sum+comb(2*eta,eta,exact=True)
        else:
            sum=sum+2*comb(2*eta,eta+i,exact=True)
    return sum/(2**(2*eta))

def pbin(eta,i):
    # centered binomial distribution probabilities
    return comb(2*eta,eta+i)/(2**(2*eta))

def quentropy(A,scale, incomplete=True):
    # scaled entropy. This function automatically completes the input set to 1; should this lead to errors due to rounding, use incomplete=False. Is designed to scale with the scaling factor of the binomial coefficient.
    if scale==0:
        return 0
    for i in range(len(A)):
        A[i]=A[i]/scale
    if incomplete:
        A=A+[1-sum(A)]
    return scale*entropy(A,base=2)

def wentropy(A,scale, incomplete=True):
    B=[A[0]]
    for i in range(1,len(A)):
        B+=[A[i]]
        B+=[A[i]]
    return quentropy(B,scale,incomplete=incomplete)

In [4]:
def uniSearch(eta):
    # creates a uniformly distributed search space
    List=(eta+1)*[1/(2*eta+1)]
    while len(List)<5:
        List+=[0]
    return List

def etaSearch(eta):
    w=max(5,eta+1)*[0]
    c=max(5,eta+1)*[0]
    c=2*sum(c)-c[0]
    for i in range(0,eta+1):
        w[i]=pbin(eta,i)
    return w

#Ternary Search spaces from May (Crypto '21)
Maydist=[
    [0.7,0.15,0,0,0],
    [0.625,0.1875,0,0,0],
    [0.559,0.2205,0,0,0],
    [0.5,0.25,0,0,0],
    [0.38,0.31,0,0,0],
    [0.333,0.3335,0,0,0]
]

def levelbib(bib, level):
    #returns all epsilon entries of a specific level
    out={}
    for i in [10,20,21,22,30,31,32,33]:
        out[i]=bib[i,level]
    return out

In [5]:
def SearchSpace(w):
    # given an input list w, this function returns the entropy of a search space where, for entries from the scale [-eta,eta] (for eta=len(w)+1), an entry i appears relatively w[i] many times
    i=len(w)
    v=[w[0]]
    for i in range(1,i):
        v=v+[w[i]]
        v=v+[w[i]]
    return entropy(v,base=2)

def Representations(w,eps):
    # this function calculates, given a previous level search space list w and an epsilon bib eps, the amount of representations from an entry of upper search spaces with entries from lower levels that are distributed according to the eps bib
    rep0=quentropy([eps[10],eps[10],eps[20],eps[20],eps[30],eps[30]],w[0])
    rep1=quentropy([eps[21],eps[21],eps[31],eps[31],(w[1]-2*eps[21]-2*eps[31])/2,(w[1]-2*eps[21]-2*eps[31])/2],w[1],incomplete=False)
    rep2=quentropy([eps[22],eps[22],eps[32],eps[32]],w[2])
    rep3=quentropy([eps[33],eps[33],(w[3]-2*eps[33])/2,(w[3]-2*eps[33])/2],w[3],incomplete=False)
    return (2*rep3+2*rep2+2*rep1+rep0)

# the cw functions calculate, according to the previous search space and an epsilon bib eps, the search space for the current level
def cw1(pw, ce):
    return (pw[1]+2*pw[2]+pw[3]-2*ce[33]-2*ce[32]-2*ce[31]+2*ce[10]-4*ce[22])/2

def cw2(pw, ce):
    return (pw[3]+2*pw[4]+2*ce[20]+2*ce[21]+2*ce[22]+2*ce[31]-2*ce[33])/2

def cw3(pw, ce):
    return (2*ce[33]+2*ce[32]+2*ce[31]+2*ce[30])/2

def cw(pw, ce):
    c1=cw1(pw, ce)
    c2=cw2(pw, ce)
    c3=cw3(pw, ce)
    c0=1-2*(c1+c2+c3)
    return [c0,c1,c2,c3,0]

In [6]:
#Evaluate_T and Evaluate_L both take an initial distribution as well as an epsilon dictionary bib and calculate each level's respective Time (or space) to create any search list of said level.

def Evaluate(dist, level, bib, retT=True, retL=True):
    T=[0]
    L=[0]
    w=[dist]
    S=[SearchSpace(w[0])]
    R=[1]
    for j in range(1,level):
        lbib=levelbib(bib,j)
        w+=[cw(w[j-1],lbib)]
        T+=[0]
        S+=[SearchSpace(w[j])]
        R+=[Representations(w[j-1],lbib)]
        L+=[S[j]-R[j]]
        if j==level-1:
            R=R+[0]
            L=L+[S[j]/2]
            T=T+[S[j]/2]
            if isnan(R[j]) or isinf(R[j]):
                T[j]=-R[j]
            elif isnan(R[level]) or isinf(R[level]):
                T[j]=R[level]
            else:
                T[j]=2*L[level]-R[j]
        if j>1:
            if isnan(R[j]) or isinf(R[j]):
                T[j-1]=-R[j]
            elif isnan(R[j-1]) or isinf(R[j-1]):
                T[j-1]=-R[j-1]
            else:
                T[j-1]=2*L[j]-(R[j-1]-R[j])
        elif isnan(R[1]) or isinf(R[1]):
            T[0]=-R[1]
        else:
            T[0]=max(0,2*L[1]-1)
    for j in range(level):
        T[j]=max(T[j],L[j+1])
    T[level]=0
    if retT and retL:
        return [T,L]
    elif retT:
        return T
    elif retL:
        return L
    return []

def Evaluate_T(dist, level, bib):
    return Evaluate(dist, level, bib, retT=True, retL=False)

def Evaluate_L(dist, level, bib):
    return Evaluate(dist, level, bib, retT=False, retL=True)

def Evaluate_bib(bib):
    for [eta,level] in bib:
        [T,L]=Evaluate(etaPermSearch(eta,range(4,eta+1)),level,bib[eta,level])
        print("------------- eta = " + str(eta) + " , level = " + str(level) + " -------------")
        print("runtime is " + str(max(T)))
        print("memory is " + str(max(L)))
        print()

In [ ]:
#Evaluate for T(0.3)
Evaluate_T(Maydist[0],4,ternarybib[0.3, 4])

In [ ]:
#Evaluate for T(0.375)
Evaluate_T(Maydist[1],5,ternarybib[0.375, 5])

In [ ]:
#Evaluate for T(0.441)
Evaluate_T(Maydist[2],6,ternarybib[0.441, 6])

In [ ]:
#Evaluate for T(0.5)
Evaluate_T(Maydist[3],5,ternarybib[0.5, 5])

In [ ]:
#Evaluate for T(0.62)
Evaluate_T(Maydist[4],6,ternarybib[0.62, 6])

In [ ]:
#Evaluate for T(0.667)
Evaluate_T(Maydist[5],6,ternarybib[0.667, 6])

In [ ]:
#Evaluate for U(1) (Same as T(0.5))
Evaluate_T(Maydist[3],5,ternarybib[0.5, 5])

In [ ]:
#Evaluate for B(2)
Evaluate_T(etaSearch(2),7,binomialbib[2, 7])

In [ ]:
#Evaluate for B(3)
Evaluate_T(etaSearch(3),7,binomialbib[3, 7])

In [ ]:
#Evaluate for B(1) (~Same as T(0.667))
Evaluate_T(Maydist[5],6,ternarybib[0.667, 6])

In [ ]:
#Evaluate for U(2)
Evaluate_T(uniSearch(2),8,uniformbib[2, 8])

In [ ]:
#Evaluate for U(3)
Evaluate_T(uniSearch(3),6,uniformbib[3, 6])